<a href="https://colab.research.google.com/github/ElmPartners/Public/blob/master/Mortgage_Optimizer_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mortgage sizing
We have the following situation:
* Personal Real-estate asset worth $H_0$, with mean $\mu_H$ and vol $\sigma_H$
* Investment portfolio worth $P_0$ with mean $\mu_P$ and vol $\sigma_P$
* A risk-free asset $R$ with fixed rate $r_f$
* Subsistence wealth level of $w_S$
* CRRA risk-aversion $\gamma$

In a 1-period world from $t=0 \rightarrow T$ we want to decide how much mortgage $M$ to take out with rate $r_M$, under two major cases:
1. M is non-recourse
2. M is full recourse

### 1. Non-recourse Case
<font size = "3">
$$ w_t = R_t + H_t + P_t^* - M_t^* $$
    
$$ R_T = R_0 e^{r_f T} $$

$$ H_T = H_0 e^{(\mu_H-\frac{1}{2}\sigma_H^2)T + \sigma_H W_H(T)} $$

$$ P_T^* = P_0^* e^{(\mu_P-\frac{1}{2}\sigma_P^2)T + \sigma_P W_P(T)} $$

$$ M_T^* = min[H_T, M_0^* e^{r_M T}] $$

$$ PDF(W_H, W_P) = \frac{1}{2\pi \sqrt{1-\rho^2}} e^{-\frac{W_H^2 - 2\rho W_H W_P + W_P^2}{2(1-\rho^2)}}$$  

$$ u(w) = \frac{max[0,(w-w_S)]^{1-\gamma} - 1}{1 - \gamma} $$
</font>

$H_0$ is fixed, we want to solve for $M_0^*$ and $P_0^*$ which maximizes $E[u(w_T)]$, and we do so by direct numerical integration:


In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Global inputs
n_pts = 50
rand_seed = 1


# The interactive main event
def inter_optimize(gamma=2, rf=0, rM=2, T=1, muH=3, sigH=6, muP=4.5, sigP=8.5, rho=0.6, H0=0.7, wS=0):
    # Rescale sliders
    rf /= 100
    rM /= 100
    muH /= 100
    sigH /= 100
    muP /= 100
    sigP /= 100
    
    # Iterate over different values of P* and M*
    Pvec = np.linspace(0, 1.5, 15, endpoint=False)
    Mvec = np.linspace(0, H0, int(np.round(H0 / 0.1, 0)), endpoint=False)
    E_u = np.zeros((len(Pvec), len(Mvec)))

    for i, P0 in enumerate(Pvec):
        for j, M0 in enumerate(Mvec):
            # Don't allow any non-mortgage leverage
            R0 = (1 - H0 - P0 + M0)
            if R0 < -0.01:
                E_u[i, j] = np.nan
                continue

            # vectorized numerical integration
            state_vec = np.linspace(-3, 3, n_pts, endpoint=True)
            xmat = np.ones((n_pts, n_pts)) * state_vec
            ymat = np.transpose(xmat)

            pdf = (1 / (2 * np.pi * pow(1 - pow(rho, 2), 0.5))) * \
                np.exp(-(np.power(xmat, 2) - 2 * rho * xmat * ymat + np.power(ymat, 2)) / (2 * (1 - pow(rho, 2))))
            R_T = R0 * np.exp(rf * T)
            H_T = H0 * np.exp((muH - 0.5 * pow(sigH, 2)) * T + sigH * pow(T, 0.5) * xmat)
            P_T = P0 * np.exp((muP - 0.5 * pow(sigP, 2)) * T + sigP * pow(T, 0.5) * ymat)
            M_T = np.minimum(H_T, M0 * np.exp(rM * T))
            w = R_T + H_T + P_T - M_T
            u = (np.power(np.maximum(np.finfo(float).eps, (w - wS)), 1 - gamma) - 1) / (1 - gamma) - \
                (pow(1 - wS, 1 - gamma) - 1) / (1 - gamma)

            E_u[i, j] = np.sum(pdf * u) / np.sum(pdf)

    # Display plots
    for m in range(0, len(Mvec)):
        plt.plot(Pvec, E_u[:, m], label='M0={}'.format(Mvec[m]), marker='*', ls=':')
    plt.xlabel('P0')
    plt.ylabel('E[u]')
    plt.grid('y')
    plt.legend()
    plt.gcf().set_size_inches(6,6)
    plt.show()


In [2]:
# Interactive app
from ipywidgets import interactive, FloatSlider, BoundedFloatText, interactive_output, HBox

w = interactive(inter_optimize, gamma=(1.25, 4, 0.25), rf=(0, 5, 0.25), rM=(0, 5, 0.25), T=(1,10), muH=(0,10,0.25), 
                sigH=(1,20,0.5), muP=(0,10,0.25), sigP=(1,25,0.5), rho=(0,0.99,0.01), H0=(0,1,0.1), wS=(0,0.9,0.05))
w

interactive(children=(FloatSlider(value=2.0, description='gamma', max=4.0, min=1.25, step=0.25), FloatSlider(v…